In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%pylab inline
import seaborn as sns

base_path = 'F:\Study\weiboPredict\data'

print('读取源微博...')
origin = pd.read_csv(base_path + '\sample_origin.csv',
                     index_col='w_id',
                     sep='\001',
                     encoding='utf-8',
                     quoting=3)
print('读取转发...')
repo = pd.read_csv(base_path + '\\sample_repo.csv',
                   sep='\001',
                   encoding='utf-8',
                   quoting=3).fillna(value='', axis=1)
 
# 只取datetime中的time
origin.time = [t.time() for t in pd.to_datetime(origin.time)]

Populating the interactive namespace from numpy and matplotlib


读取源微博...


读取转发...


In [23]:
# 系统抽样
def systematic_sampling(dataMat, num):
    import random
    k = int(len(dataMat)/num)
    
    samples = [random.sample(dataMat[i*k:(i+1)*k], 1)[0] for i in range(num)]
    return samples

In [24]:
test_size = int(0.2*origin.shape[0])
test_index = systematic_sampling(list(range(origin.shape[0])),test_size)
 
test_set = pd.DataFrame(origin.sort_values('repo_num_1w').iloc[test_index])
test_set.describe()

,author_id,repo_num_1w,mean_depth,90p_depth,max_depth
count,5.199000e+03,5199.000000,5199.000000,5199.000000,5199.000000
mean,5.278539e+06,556.556453,1.340614,1.878823,3.428159
std,2.384068e+06,399.471059,0.506703,1.247518,2.093290
min,1.249800e+04,75.000000,0.500000,1.000000,1.000000
25%,3.531765e+06,193.000000,1.010638,1.000000,2.000000
50%,5.736613e+06,416.000000,1.112211,2.000000,3.000000
75%,7.362190e+06,922.000000,1.517495,2.000000,4.000000
max,8.379984e+06,1494.000000,6.696784,12.000000,16.000000


In [27]:
rest_sample = origin.drop(test_set.index,axis=0)
rest_sample.describe()

,author_id,repo_num_1w,mean_depth,90p_depth,max_depth
count,2.080000e+04,20800.000000,20800.000000,20800.000000,20800.000000
mean,5.242679e+06,556.736346,1.345381,1.891875,3.460817
std,2.384347e+06,399.614785,0.520390,1.267368,2.120873
min,6.824000e+03,75.000000,0.500000,1.000000,1.000000
25%,3.470838e+06,193.000000,1.011940,1.000000,2.000000
50%,5.687616e+06,416.000000,1.120104,2.000000,3.000000
75%,7.298946e+06,922.000000,1.518607,2.000000,4.000000
max,8.379984e+06,1506.000000,8.063518,17.000000,22.000000


In [29]:
vali_size = int(0.2*rest_sample.shape[0])
vali_index = systematic_sampling(list(range(rest_sample.shape[0])),vali_size)
 
vali_set = pd.DataFrame(rest_sample.sort_values('repo_num_1w').iloc[vali_index])
vali_set.describe()

,author_id,repo_num_1w,mean_depth,90p_depth,max_depth
count,4.160000e+03,4160.000000,4160.000000,4160.000000,4160.000000
mean,5.226075e+06,556.734615,1.341161,1.875240,3.455529
std,2.394741e+06,399.647931,0.503167,1.200129,2.082211
min,4.261800e+04,75.000000,0.500000,1.000000,1.000000
25%,3.464099e+06,193.000000,1.012076,1.000000,2.000000
50%,5.645189e+06,416.500000,1.123420,2.000000,3.000000
75%,7.298946e+06,922.250000,1.517964,2.000000,4.000000
max,8.361536e+06,1497.000000,5.993808,11.000000,18.000000


In [31]:
train_set = rest_sample.drop(vali_set.index,axis=0)
train_set.describe()

,author_id,repo_num_1w,mean_depth,90p_depth,max_depth
count,1.664000e+04,16640.000000,16640.000000,16640.000000,16640.000000
mean,5.246830e+06,556.736779,1.346436,1.896034,3.462139
std,2.381796e+06,399.618508,0.524617,1.283629,2.130489
min,6.824000e+03,75.000000,0.500000,1.000000,1.000000
25%,3.474536e+06,193.000000,1.011924,1.000000,2.000000
50%,5.699560e+06,416.000000,1.119256,2.000000,3.000000
75%,7.298946e+06,922.000000,1.518826,2.000000,4.000000
max,8.379984e+06,1506.000000,8.063518,17.000000,22.000000


In [32]:
train_repo = repo[repo['w_id'].isin(train_set.index)]
test_repo = repo[repo['w_id'].isin(test_set.index)]
vali_repo = repo[repo['w_id'].isin(vali_set.index)]

In [38]:
train_set.to_csv(base_path+'\\samples\\origin_train.csv',
                sep='\001',
                columns = ['author_id','time','repo_num_1w','max_depth','90p_depth','mean_depth','content'],
                index_label='w_id',
                encoding='utf-8',
                quoting=3)
train_repo.to_csv(base_path+'\\samples\\repo_train.csv',
                sep='\001',
                index=False,
                encoding='utf-8',
                quoting=3)

vali_set.to_csv(base_path+'\\samples\\origin_vali.csv',
                sep='\001',
                columns = ['author_id','time','repo_num_1w','max_depth','90p_depth','mean_depth','content'],
                index_label='w_id',
                encoding='utf-8',
                quoting=3)
vali_repo.to_csv(base_path+'\\samples\\repo_vali.csv',
                sep='\001',
                index=False,
                encoding='utf-8',
                quoting=3)

test_set.to_csv(base_path+'\\samples\\origin_test.csv',
                sep='\001',
                columns = ['author_id','time','repo_num_1w','max_depth','90p_depth','mean_depth','content'],
                index_label='w_id',
                encoding='utf-8',
                quoting=3)
test_repo.to_csv(base_path+'\\samples\\repo_test.csv',
                sep='\001',
                index=False,
                encoding='utf-8',
                quoting=3)